In [1]:
import datetime
import numpy as np
import matplotlib.pylab as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [67]:
EPOCH_NUM = 10
HIDDEN_SIZE = 5
BATCH_SIZE = 100 # 分割した時系列をいくつミニバッチに取り込むか
INPUT_LENGTH = 20 # ミニバッチで分割する時系列数

# 教師データ
train_data = np.array([np.sin(i*2*np.pi/50) for i in range(50)]*10)

# 教師データを変換
train_x, train_t = [], []
for i in range(0, len(train_data)-INPUT_LENGTH):
    train_x.append(train_data[i:i+INPUT_LENGTH])
    train_t.append(train_data[i+INPUT_LENGTH])
train_x = np.array(train_x, dtype="float32")
train_t = np.array(train_t, dtype="float32")
N = len(train_x)

train_x=train_x.reshape(N,INPUT_LENGTH,1)#これしないと(data_num, sep_len, )となってしまう
train_t=train_t.reshape(N,1)
print(train_x.shape)
print(train_t.shape)

(480, 20, 1)
(480, 1)


クラス定義
lstmの出力が、シーケンスででてくるけど、欲しいのは最終セルの出力だけなんだよなあ

In [92]:
class SinLstm(nn.Module):
    def __init__(self, seq_size, hidden_size, out_size):
        # クラスの初期化
        # :param seq_size: 入力時系列のサイズ
        # :param hidden_size: 隠れ層のサイズ
        # :param out_size: 出力層のサイズ
        super(SinLstm, self).__init__()
        self.lstm = torch.nn.LSTM(hidden_size= hidden_size,input_size=1,batch_first=True)
        self.dense= torch.nn.Linear(hidden_size, out_size)
        self.hidden_size = hidden_size

    def forward(self, x):
        # x: 入力時系列
        outputs, (h_n,c_n) = self.lstm(x)
        h_n=h_n.squeeze()
        y=self.dense(h_n)
        return y

    def reset(self):
        # メモリの初期化
        self.hidden = (Variable(torch.zeros(1, 1, self.hidden_size)), Variable(torch.zeros(1, 1, self.hidden_size))) # h, c

In [93]:
model = SinLstm(seq_size=INPUT_LENGTH, hidden_size=HIDDEN_SIZE, out_size=1)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())
print(model)

SinLstm(
  (lstm): LSTM(1, 5, batch_first=True)
  (dense): Linear(in_features=5, out_features=1, bias=True)
)


In [96]:
# 学習開始
print("Train")
for epoch in range(EPOCH_NUM):
    # ミニバッチ学習
    x, t = [], []
    #  ミニバッチ学習データとして、BATCH_SIZE個用意する
    for i in range(BATCH_SIZE):
        index = np.random.randint(0, N) # ランダムな箇所
        x.append(train_x[index]) # INPUT_LENGTH分の時系列を取り出す
        t.append(train_t[index])
    x = np.array(x, dtype="float32")
    t = np.array(t, dtype="float32")
    x = Variable(torch.from_numpy(x))
    t = Variable(torch.from_numpy(t))
    total_loss = 0
    model.reset() # メモリの初期化
    y = model(x)#xのshapeは(batch_size, seq_len, input_dim)となっていなければいけない
    loss = criterion(y, t)
    loss.backward()
    total_loss += loss.data
    optimizer.step()
    print(total_loss)

Train
tensor(0.5403)
tensor(0.4474)
tensor(0.5103)
tensor(0.5417)
tensor(0.4580)
tensor(0.4837)
tensor(0.4901)
tensor(0.4697)
tensor(0.4334)
tensor(0.5226)
